# 1.线性增强

In [ ]:
from PIL import Image


def linear_enhancement(image_path, alpha, beta):
    # 打开图像
    image = Image.open(image_path)

    # 获取图像的宽度和高度
    width, height = image.size

    # 创建一个新的图像对象，用于存储增强后的图像
    enhanced_image = Image.new("RGB", (width, height))

    # 遍历图像的每个像素点，并进行线性增强
    for x in range(width):
        for y in range(height):
            # 获取原始像素值
            r, g, b = image.getpixel((x, y))

            # 进行线性增强计算
            r_enhanced = int(alpha * r + beta)
            g_enhanced = int(alpha * g + beta)
            b_enhanced = int(alpha * b + beta)

            # 将增强后的像素值设置到新图像中
            enhanced_image.putpixel((x, y), (r_enhanced, g_enhanced, b_enhanced))

    # 返回增强后的图像
    return enhanced_image


# 使用示例
input_image_path = "图片样例.jpg"  # 输入图像路径
output_image_path = "线性增强后3.jpg"  # 输出图像路径
alpha = 0.8  # 增强系数
beta = 30  # 增强偏移量

enhanced_image = linear_enhancement(input_image_path, alpha, beta)
enhanced_image.save(output_image_path)


<div style="display: flex; justify-content: center;">
  <div style="text-align: center; margin-right: 10px;">
    <img src="1.线性增强/图片样例.jpg" alt="Image 1" style="width: 120%;">
    <p>Original Image</p>
  </div>
  
  <div style="text-align: center; margin-right: 10px;">
    <img src="1.线性增强/线性增强后.jpg" alt="Image 2" style="width: 120%;">
    <p>参数1</p>
  </div>

  <div style="text-align: center; margin-right: 10px;">
    <img src="1.线性增强/线性增强后2.jpg" alt="Image 2" style="width: 120%;">
    <p>参数2</p>
  </div>
</div>

***

# 2.高斯滤波

In [ ]:
import numpy as np
import cv2


def gaussian_blur(image_path, kernel_size, sigma):
    # 读取图像
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # 以灰度模式读取图像

    # 创建高斯核
    kernel = create_gaussian_kernel(kernel_size, sigma)

    # 执行滤波操作
    blurred_image = convolve(image, kernel)

    # 返回滤波后的图像
    return blurred_image


def create_gaussian_kernel(kernel_size, sigma):
    # 计算高斯核的大小
    size = int((kernel_size - 1) / 2)

    # 创建高斯核
    kernel = np.fromfunction(
        lambda x, y: (1 / (2 * np.pi * sigma ** 2)) * 
        np.exp(-((x - size) ** 2 + (y - size) ** 2) / 
               (2 * sigma ** 2)), (kernel_size, kernel_size))

    # 归一化高斯核
    kernel /= np.sum(kernel)

    return kernel


def convolve(image, kernel):
    # 获取图像的尺寸
    height, width = image.shape

    # 获取高斯核的尺寸
    kernel_size = kernel.shape[0]

    # 计算高斯核的大小
    size = int((kernel_size - 1) / 2)

    # 创建一个新的图像对象（空白），用于存储滤波后的图像
    blurred_image = np.zeros((height, width), dtype=np.uint8)
                                             # uint8:无符号八位整数，作为数组的数据类型

    # 执行卷积操作（最边上的像素点不进行卷积）
    for row in range(size, height - size):
        for col in range(size, width - size):
            # 提取当前像素周围的图像块
            block = image[row - size: row + size + 1, col - size: col + size + 1]

            # 进行卷积计算
            convolved_value = np.sum(block * kernel)

            # 将卷积结果存储到新图像中
            blurred_image[row, col] = int(convolved_value)

    return blurred_image


# 使用示例
input_image_path = "img.jpg"  # 输入图像路径
output_image_path = "gaussian100.jpg"  # 输出图像路径
kernel_size = 5  # 高斯核大小
sigma = 100.0  # 高斯核标准差

blurred_image = gaussian_blur(input_image_path, kernel_size, sigma)
cv2.imwrite(output_image_path, blurred_image)


<div style="display: flex; justify-content: center;">
  <div style="text-align: center; margin-right: 10px;">
    <img src="2.高斯滤波/img.jpg" alt="Image 1" style="width: 120%;">
    <p>Original Image</p>
  </div>
  
  <div style="text-align: center; margin-right: 10px;">
    <img src="2.高斯滤波/gaussian1.jpg" alt="Image 2" style="width: 120%;">
    <p>sigma = 1</p>
  </div>

  <div style="text-align: center; margin-right: 10px;">
    <img src="2.高斯滤波/gaussian10.jpg" alt="Image 2" style="width: 120%;">
    <p>sigma = 10</p>
  </div>

  <div style="text-align: center; margin-right: 10px;">
    <img src="2.高斯滤波/gaussian100.jpg" alt="Image 2" style="width: 120%;">
    <p>sigma = 100</p>
  </div>  
</div>

***

# 3.高斯锐化

In [ ]:
import numpy as np
from scipy.ndimage import convolve
import cv2

def gaussian_sharpen(image, sigma, strength):
    # 创建高斯核
    size = int(2 * np.ceil(3 * sigma) + 1)
    kernel = np.fromfunction(lambda x, y: (1/(2*np.pi*sigma**2)) * 
                             np.exp(-((x-size//2)**2 + (y-size//2)**2)/(2*sigma**2)), 
                             (size, size))
    kernel /= np.sum(kernel)  # 归一化

    # 对图像进行卷积操作
    blurred = convolve(image, kernel)

    # 高斯锐化：原始图像加上锐化图像
    sharpened = image + (image - blurred) * strength

    # 将像素值限制在0到255之间
    sharpened = np.clip(sharpened, 0, 255).astype(np.uint8)

    return sharpened

# 读取图像
image = cv2.imread('img.jpg', 0)  # 以灰度模式读取图像

# 调用高斯锐化函数
sharpened_image = gaussian_sharpen(image, 2.0, 1.5)

# 显示原始图像和锐化后的图像
cv2.imshow('Original Image', image)
cv2.imshow('Sharpened Image', sharpened_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

<div style="display: flex; justify-content: center;">
  <div style="text-align: center; margin-right: 20px;">
    <img src="3.高斯锐化/original_image.jpg" alt="Image 1" style="width: 110%;">
    <p>Original Image</p>
  </div>
  
  <div style="text-align: center; margin-right: 20px;">
    <img src="3.高斯锐化/sharpened_image.jpg" alt="Image 2" style="width: 110%;">
    <p>Sharpened Image</p>
  </div>
</div>

***

# 4.插值放大

In [ ]:
import os
import cv2
import sys
sys.stdout.reconfigure(encoding='utf-8')

# 获取当前脚本文件的路径
current_dir = os.path.dirname(os.path.abspath(__file__))
print(current_dir)

# 将当前工作目录设置为代码文件所在的目录
os.chdir(current_dir)

# 读取输入图像
input_image = cv2.imread('..\img.jpg')

# 定义放大比例
scale_percent = 2  # 放大比例为2倍

# 最近邻插值
nearest_neighbor = cv2.resize(input_image, None, fx=scale_percent, fy=scale_percent, 
                              interpolation=cv2.INTER_NEAREST)

# 双线性插值
bilinear = cv2.resize(input_image, None, fx=scale_percent, fy=scale_percent, 
                      interpolation=cv2.INTER_LINEAR)

# 三次样条插值
bicubic = cv2.resize(input_image, None, fx=scale_percent, fy=scale_percent, 
                     interpolation=cv2.INTER_CUBIC)

# 保存放大后的图像到本地
cv2.imwrite('nearest_neighbor.jpg', nearest_neighbor)
cv2.imwrite('bilinear.jpg', bilinear)
cv2.imwrite('bicubic.jpg', bicubic)

<div style="display: flex; justify-content: center;">
  <div style="text-align: center; margin-right: 20px;">
    <img src="4.插值放大/nearest_neighbor.jpg" alt="Image 1" style="width: 110%;">
    <p>Nearest Neighbor Interpolation</p>
  </div>
  
  <div style="text-align: center; margin-right: 20px;">
    <img src="4.插值放大/bilinear.jpg" alt="Image 2" style="width: 110%;">
    <p>Bilinear Interpolation</p>
  </div>
  
  <div style="text-align: center;">
    <img src="4.插值放大/bicubic.jpg" alt="Image 3" style="width: 110%;">
    <p>Bicubic Interpolation</p>
  </div>
</div>

***

# 5.图像变换

In [ ]:
import os
import cv2
import numpy as np
import sys
sys.stdout.reconfigure(encoding='utf-8')

# 获取当前脚本文件的路径
current_dir = os.path.dirname(os.path.abspath(__file__))
print(current_dir)

# 将当前工作目录设置为代码文件所在的目录
os.chdir(current_dir)

# 读取图像
image = cv2.imread('..\img.jpg')

# 平移变换
translation_matrix = np.float32([[1, 0, 100], [0, 1, 50]])  # 平移变换矩阵
translated_image = cv2.warpAffine(image, translation_matrix, (image.shape[1], image.shape[0]))

# 旋转变换
rotation_matrix = cv2.getRotationMatrix2D((image.shape[1] / 2, image.shape[0] / 2), 30, 1)  # 旋转变换矩阵
rotated_image = cv2.warpAffine(image, rotation_matrix, (image.shape[1], image.shape[0]))

# 相似变换
similarity_matrix = cv2.getRotationMatrix2D((image.shape[1] / 2, image.shape[0] / 2), 45, 0.8)  # 相似变换矩阵
similar_image = cv2.warpAffine(image, similarity_matrix, (image.shape[1], image.shape[0]))

# 仿射变换
pts1 = np.float32([[50, 50], [200, 50], [50, 200]])  # 原图像上的三个点
pts2 = np.float32([[50, 100], [200, 50], [100, 250]])  # 目标图像上的对应三个点
affine_matrix = cv2.getAffineTransform(pts1, pts2)  # 仿射变换矩阵
affine_image = cv2.warpAffine(image, affine_matrix, (image.shape[1], image.shape[0]))

# 投影变换
pts1 = np.float32([[56, 65], [368, 52], [28, 387], [389, 390]])  # 原图像上的四个点
pts2 = np.float32([[0, 0], [300, 0], [0, 300], [300, 300]])  # 目标图像上的对应四个点
perspective_matrix = cv2.getPerspectiveTransform(pts1, pts2)  # 投影变换矩阵
perspective_image = cv2.warpPerspective(image, perspective_matrix, (image.shape[1], image.shape[0]))

# 将图像移动到中央位置
center_x = image.shape[1] // 2
center_y = image.shape[0] // 2
translate_x = center_x - image.shape[1] // 2
translate_y = center_y - image.shape[0] // 2

# 创建独立的平移变换矩阵
translation_matrix_translated = np.float32([[1, 0, translate_x], [0, 1, translate_y]])
translation_matrix_rotated = np.float32([[1, 0, translate_x], [0, 1, translate_y]])
translation_matrix_similar = np.float32([[1, 0, translate_x], [0, 1, translate_y]])
translation_matrix_affine = np.float32([[1, 0, translate_x], [0, 1, translate_y]])
translation_matrix_perspective = np.float32([[1, 0, translate_x], [0, 1, translate_y]])

# 应用平移变换
translated_image = cv2.warpAffine(translated_image, translation_matrix_translated, (image.shape[1], image.shape[0]))
rotated_image = cv2.warpAffine(rotated_image, translation_matrix_rotated, (image.shape[1], image.shape[0]))
similar_image = cv2.warpAffine(similar_image, translation_matrix_similar, (image.shape[1], image.shape[0]))
affine_image = cv2.warpAffine(affine_image, translation_matrix_affine, (image.shape[1], image.shape[0]))
perspective_image = cv2.warpAffine(perspective_image, translation_matrix_perspective, (image.shape[1], image.shape[0]))

# 保存图像
cv2.imwrite('translated_image.jpg', translated_image)
cv2.imwrite('rotated_image.jpg', rotated_image)
cv2.imwrite('similar_image.jpg', similar_image)
cv2.imwrite('affine_image.jpg', affine_image)
cv2.imwrite('perspective_image.jpg', perspective_image)

<div style="display: flex; justify-content: center;">
  <div style="text-align: center; margin-right: 20px;">
    <img src="img.jpg" alt="Image 1" style="width: 110%;">
    <p>Original Image</p>
  </div>
  
  <div style="text-align: center; margin-right: 20px;">
    <img src="5.图像变换/similar_image.jpg" alt="Image 2" style="width: 110%;">
    <p>Similar Image</p>
  </div>
  
  <div style="text-align: center;">
    <img src="5.图像变换/translated_image.jpg" alt="Image 3" style="width: 110%;">
    <p>Translated Image</p>
  </div>
</div>

<div style="display: flex; justify-content: center;">
  <div style="text-align: center; margin-right: 20px;">
    <img src="5.图像变换/affine_image.jpg" alt="Image 1" style="width: 110%;">
    <p>Affine Image</p>
  </div>
  
  <div style="text-align: center; margin-right: 20px;">
    <img src="5.图像变换/perspective_image.jpg" alt="Image 2" style="width: 110%;">
    <p>Perspective Image</p>
  </div>
  
  <div style="text-align: center;">
    <img src="5.图像变换/rotated_image.jpg" alt="Image 3" style="width: 110%;">
    <p>Rotated Image</p>
  </div>
</div>

***

# 6.前后映射

In [ ]:
import os
import cv2
import numpy as np
import sys
sys.stdout.reconfigure(encoding='utf-8')

# 获取当前脚本文件的路径
current_dir = os.path.dirname(os.path.abspath(__file__))
print(current_dir)

# 将当前工作目录设置为代码文件所在的目录
os.chdir(current_dir)

# 向前映射函数
def forward_mapping_rotation(image, angle):
    height, width = image.shape[:2]
    center = (width / 2, height / 2)
    
    # 计算旋转矩阵
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    
    # 进行前向映射
    rotated_image = np.zeros_like(image)
    for y in range(height):
        for x in range(width):
            # 计算旋转后的坐标
            new_coords = np.dot(rotation_matrix, [x, y, 1])
            new_x, new_y = new_coords[:2]
            
            # 边界检查
            if 0 <= new_x < width and 0 <= new_y < height:
                # 将像素值赋给旋转后的位置
                rotated_image[int(new_y), int(new_x),:] = image[y, x, :]
    
    return rotated_image

# 向后映射函数
def inverse_mapping_rotation(image, angle):
    height, width = image.shape[:2]
    center = (width / 2, height / 2)
    
    # 计算反向旋转矩阵
    rotation_matrix = cv2.getRotationMatrix2D(center, -angle, 1.0)
    
    # 进行反向映射
    rotated_image = np.zeros_like(image)
    
    for y in range(height):
        for x in range(width):
            # 计算反向旋转后的坐标
            original_coords = np.dot(rotation_matrix, [x, y, 1])
            original_x, original_y = original_coords[:2]
            
            # 边界检查
            if 0 <= original_x < width and 0 <= original_y < height:
                # 将像素值赋给旋转前的位置
                rotated_image[y, x] = image[int(original_y), int(original_x)]
    
    return rotated_image

# 读取图像
image = cv2.imread('..\img.jpg')

# 向前映射旋转
forward_rotated_image = forward_mapping_rotation(image, 30)

# 向后映射旋转
inverse_rotated_image = inverse_mapping_rotation(image, 30)

# 显示结果
cv2.imshow('Original Image', image)
cv2.imshow('Forward Rotated Image', forward_rotated_image)
cv2.imshow('Inverse Rotated Image', inverse_rotated_image)
# cv2.imwrite('forward_rotated_image.jpg', forward_rotated_image)
# cv2.imwrite('inverse_rotated_image.jpg', inverse_rotated_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

<div style="display: flex; justify-content: center;">
  <div style="text-align: center; margin-right: 20px;">
    <img src="img.jpg" alt="Image 1" style="width: 110%;">
    <p>Original Image</p>
  </div>
  
  <div style="text-align: center; margin-right: 20px;">
    <img src="6.前后映射/forward_rotated_image.jpg" alt="Image 2" style="width: 110%;">
    <p>前向映射</p>
  </div>
  
  <div style="text-align: center;">
    <img src="6.前后映射/inverse_rotated_image.jpg" alt="Image 3" style="width: 110%;">
    <p>后向映射</p>
  </div>
</div>

<div style="display: flex; justify-content: center;">
  <div style="text-align: center; margin-right: 20px;">
    <img src="6.前后映射/image.png" alt="Image 1" style="width: 50%;">
    <p>前向映射放大后</p>
  </div>
</div>